In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import ase 
from ase.units import Bohr 
import torch
import metatensor
from metatensor import TensorMap, TensorBlock, Labels
import matplotlib.pyplot as plt
from ase.io import read 
import hickle
from mlelec.utils.metatensor_utils import labels_where

from mlelec.data.dataset import PeriodicDataset

In [3]:
device='cpu'
filename = "C2_rotated"
frames = read('dataset/periodic/rotation/C2_rotated.xyz', ':') # will be automated from filename  
for f in frames:
    f.pbc = [True, True, True]
kfock = np.load('dataset/periodic/rotation/H_k.npz'.format(filename), allow_pickle=True)
kfock=[kfock['arr_{}'.format(i)] for i in range(len(kfock))]
print(kfock[0].shape)
fock = np.load('dataset/periodic/rotation/realspaceH_nonsym.npz'.format(filename), allow_pickle=True)
fock=np.array([fock['arr_{}'.format(i)] for i in range(len(fock))])
print(fock[0].shape)

all_shifts = np.load('dataset/periodic/rotation/cell_idx_list.npz'.format(filename), allow_pickle=True)
all_shifts=np.array([all_shifts['arr_{}'.format(i)] for i in range(len(all_shifts))])
print(all_shifts[0].shape)

pyscfkfock = np.load('../examples/data/periodic/c2/results_{}/kfocks.npy'.format(filename))

#desired_shifts=[[0,0,0],[0,1,0],[1,0,0],[1,1,0],[1,-1,0],[0,2,0],[2,0,0],[2,1,0],[1,2,0],[2,2,0],[2,-1,0],[1,-2,0]]
#rotated_dataset = PeriodicDataset(frames = frames, kgrid=[4,4,1], matrices_kpoint = pyscfkfock, target=["real_translation"] ,device = device, desired_shifts=desired_shifts)

(868, 10, 10)
(99, 10, 10)
(99, 3)


In [4]:
#dir(rotated_dataset)

In [5]:
pyscfkfock.shape

(3, 16, 10, 10)

In [6]:
#rotated_dataset.desired_shifts

In [7]:
## This should happen in the ML Dataset class #TODO
## Fix orbital order 
#from mlelec.utils.twocenter_utils import fix_orbital_order
orbs = {6: [[1,0,0],[2,0,0],[2,1,1], [2,1,-1],[2,1,0]]}
#for T in rotated_dataset.matrices_translation.keys():
#    rotated_dataset.matrices_translation[T] = fix_orbital_order(rotated_dataset.matrices_translation[T], frames, orbs)


In [8]:
#withnegative_shifts = list(rotated_dataset.desired_shifts_sup.keys())
# withnegative_shifts.remove((0,0,0))
#desired_shifts = list(rotated_dataset.desired_shifts)
#desired_shifts.remove([0,0,0]) # for now we remove the zero shift until i have incorporate featuires for zero shift into the feats 

In [9]:
all_shifts.shape#[0]

(3, 99, 3)

In [10]:
import hickle
from mlelec.utils.twocenter_utils_fhiaims import _fhiaims_to_condon_shortley, check_rotation
from mlelec.utils.twocenter_utils_fhiaims import symmetrize_matrices, to_coupled_blocks
from mlelec.utils.twocenter_utils_fhiaims import _matrix_to_blocks, _to_matrix, _to_coupled_basis, _to_uncoupled_basis

In [11]:
# fock_map= {}
# for s in translated_matrices.keys():
#     #print(s)
#     fock_map[tuple(s)] = _to_coupled_basis(_matrix_to_blocks(translated_matrices[tuple(s)], frames=frames, orbitals=orbs,), orbs) # uncouple -> coupled    


In [12]:
# len(translated_matrices.keys())
# translated_matrices
len(fock.shape)

4

In [13]:
#fixed_fock=_fhiaims_to_condon_shortley(fock[:,0], frames, orbs, device=device)
fixed_fock=_fhiaims_to_condon_shortley(fock, frames, orbs, device=device)
#     matrices: Union[List[torch.tensor], torch.tensor],
#     frames: Union[ase.Atoms, List[ase.Atoms]],
#     orbitals: dict,
#     device: str = None,
# )

In [14]:
fock[:,0].shape

(3, 10, 10)

In [15]:
fixed_fock.shape

torch.Size([3, 99, 10, 10])

In [16]:
translated_matrices=dict([(tuple(all_shifts[0][i]),fixed_fock[:,i]) for i in range(len(all_shifts[0])) if np.linalg.norm(fock[:,i])>10**-3] )

In [17]:
#fixed_fock[:,0]

In [18]:
rotations=hickle.load('dataset/periodic/rotation/rotations.hickle')

In [19]:
rotations

[tensor([ 1.5078, -0.0463, -0.8321]), tensor([ 0.1443, -0.1218,  0.4463])]

In [20]:
blocks=to_coupled_blocks(translated_matrices,frames,orbs, device=device)
check_rotation(blocks,rotations,precision=10**-6)

#blocks=_to_coupled_basis(_matrix_to_blocks(fixed_fock, frames=frames, orbitals=orbs,), orbs, device=device)
#check_rotation({(0,0,0):blocks},rotations)

0 0 [0 6 1 0 6 1 0 0] tensor(4.7204e-05)
0 1 [0 6 1 0 6 2 0 0] tensor(4.7204e-06)
0 3 [0 6 2 0 6 2 0 0] tensor(6.4063e-06)
0 5 [0 6 2 1 6 2 1 0] tensor(6.4063e-06)
0 6 [0 6 2 1 6 2 1 2] tensor(6.2326e-06)
0 8 [1 6 1 0 6 2 0 0] tensor(2.2759e-06)
0 11 [-1  6  1  0  6  2  1  1] tensor(4.4230e-06)
0 14 [-1  6  2  0  6  2  1  1] tensor(2.4827e-06)
0 15 [1 6 2 1 6 2 1 0] tensor(7.1650e-06)
0 16 [1 6 2 1 6 2 1 2] tensor(4.0055e-06)
11 16 [1 6 2 1 6 2 1 2] tensor(1.0500e-06)
12 16 [1 6 2 1 6 2 1 2] tensor(1.0217e-06)
17 4 [0 6 2 0 6 2 1 1] tensor(1.0748e-06)
17 5 [0 6 2 1 6 2 1 0] tensor(2.3997e-06)
17 6 [0 6 2 1 6 2 1 2] tensor(2.5161e-06)
17 12 [1 6 2 0 6 2 0 0] tensor(1.4336e-06)
17 14 [-1  6  2  0  6  2  1  1] tensor(1.8803e-06)
17 15 [1 6 2 1 6 2 1 0] tensor(1.1478e-06)
17 16 [1 6 2 1 6 2 1 2] tensor(4.6325e-06)
18 4 [0 6 2 0 6 2 1 1] tensor(1.0227e-06)
18 5 [0 6 2 1 6 2 1 0] tensor(2.3918e-06)
18 6 [0 6 2 1 6 2 1 2] tensor(2.6614e-06)
18 8 [1 6 1 0 6 2 0 0] tensor(1.6690e-06)
18 11 [-1 

In [21]:
#blocks

In [22]:
translated_matrices#.keys())

{(0,
  0,
  0): tensor([[[-1.0165e+01,  1.8341e-02,  8.4374e-04,  1.1033e-14,  4.8257e-04,
           -2.2604e-03,  5.7346e-01, -5.3323e-01,  0.0000e+00,  9.0806e-01],
          [ 1.8341e-02, -1.0110e+00, -1.3739e-02, -8.8424e-14, -7.7214e-03,
            5.7346e-01, -5.3616e-01,  2.6260e-01,  0.0000e+00, -4.4589e-01],
          [ 8.4374e-04, -1.3739e-02, -8.8915e-01, -1.3618e-16, -2.4177e-03,
            5.3323e-01, -2.6260e-01, -2.7946e-01,  0.0000e+00, -2.3237e-01],
          [ 1.1033e-14, -8.8424e-14, -1.3618e-16, -4.3218e-01, -1.0170e-16,
            0.0000e+00,  0.0000e+00,  0.0000e+00, -2.5713e-01, -0.0000e+00],
          [ 4.8257e-04, -7.7214e-03, -2.4177e-03, -1.0170e-16, -8.9670e-01,
           -9.0806e-01,  4.4589e-01, -2.3237e-01, -0.0000e+00, -3.5125e-02],
          [-2.2604e-03,  5.7346e-01,  5.3323e-01,  0.0000e+00, -9.0806e-01,
           -1.0165e+01,  1.8341e-02, -8.4374e-04,  1.0891e-14, -4.8257e-04],
          [ 5.7346e-01, -5.3616e-01, -2.6260e-01,  0.0000e+00,  4.4

In [23]:
desired_shifts=[[0,0,0],[0,1,0],[1,0,0],[1,1,0],[1,-1,0],[0,2,0],[2,0,0],[2,1,0],[1,2,0],[2,2,0],[2,-1,0],[1,-2,0]]
desired_shifts=[tuple(i) for i in desired_shifts]

In [24]:
filtered_matrices=dict([(i, translated_matrices[i]) for i in translated_matrices.keys() if i in desired_shifts])

In [25]:
filtered_matrices

{(0,
  0,
  0): tensor([[[-1.0165e+01,  1.8341e-02,  8.4374e-04,  1.1033e-14,  4.8257e-04,
           -2.2604e-03,  5.7346e-01, -5.3323e-01,  0.0000e+00,  9.0806e-01],
          [ 1.8341e-02, -1.0110e+00, -1.3739e-02, -8.8424e-14, -7.7214e-03,
            5.7346e-01, -5.3616e-01,  2.6260e-01,  0.0000e+00, -4.4589e-01],
          [ 8.4374e-04, -1.3739e-02, -8.8915e-01, -1.3618e-16, -2.4177e-03,
            5.3323e-01, -2.6260e-01, -2.7946e-01,  0.0000e+00, -2.3237e-01],
          [ 1.1033e-14, -8.8424e-14, -1.3618e-16, -4.3218e-01, -1.0170e-16,
            0.0000e+00,  0.0000e+00,  0.0000e+00, -2.5713e-01, -0.0000e+00],
          [ 4.8257e-04, -7.7214e-03, -2.4177e-03, -1.0170e-16, -8.9670e-01,
           -9.0806e-01,  4.4589e-01, -2.3237e-01, -0.0000e+00, -3.5125e-02],
          [-2.2604e-03,  5.7346e-01,  5.3323e-01,  0.0000e+00, -9.0806e-01,
           -1.0165e+01,  1.8341e-02, -8.4374e-04,  1.0891e-14, -4.8257e-04],
          [ 5.7346e-01, -5.3616e-01, -2.6260e-01,  0.0000e+00,  4.4

## Matrix to blocks 

In [26]:
from mlelec.utils.twocenter_utils import _to_blocks, _to_matrix, _to_coupled_basis, _to_uncoupled_basis, discard_nonhermiticity

In [27]:
# Extracting matrices out from the dataset for now 
matrices = {}
for s in list(filtered_matrices.keys()): # positive translations only 
    matrices[tuple(s)] = filtered_matrices[tuple(s)].to(device)
    # NORMALIZING the translated matrices for each structure - not sure this is for training - SKIP if not needed
    # matrices[tuple(s)] = matrices[tuple(s)]/torch.linalg.norm(matrices[tuple(s)], axis = (1,2) ) [:,None, None]


In [28]:
#matrices[(0,0,0)]

In [29]:
target_blocks = {}
target_blocks = {}
target_coupled_blocks = {}
for s in list(filtered_matrices.keys()):
    
    target_blocks[tuple(s)] = _to_blocks(matrices[tuple(s)], frames=frames, orbitals=orbs, NH=True, device=device) # matrix -> uncoupled
    # Dont forget NH = True
    target_coupled_blocks[tuple(s)] = _to_coupled_basis(target_blocks[tuple(s)], orbs, skip_symmetry=True,device=device) # uncouple -> coupled
    # Dont forget skip_symmetry = True

/home/hanna/data/Hamiltonian/jigyasa/mlelec/fhiaims/mlelec/utils/twocenter_utils.py:162: UserWarning: Matrix is neither hermitian nor antihermitian - attempting to use _toblocks for NH
  warnings.warn(


In [30]:
from collections import defaultdict
blocks = []
tmap_keys = []
for translation in target_coupled_blocks.keys():
    for i, (key, block) in enumerate(target_coupled_blocks[translation].items()):
        blocks.append(block.copy())
        x = list(key)
        x.extend(translation)
        tmap_keys.append(x)
keys = Labels(next(iter(target_coupled_blocks.values())).keys.names+["cell_shift_a", "cell_shift_b", "cell_shift_c"], values=np.asarray(tmap_keys))
target = TensorMap(keys=keys, blocks=blocks)

target = target.keys_to_samples('cell_shift_a')
target = target.keys_to_samples('cell_shift_b')
target = target.keys_to_samples('cell_shift_c')


In [31]:
target[0].samples.values
target_coupled_blocks.keys()
#desired_shifts

dict_keys([(0, 0, 0), (0, 1, 0), (0, 2, 0), (1, -2, 0), (1, -1, 0), (1, 0, 0), (1, 1, 0), (1, 2, 0), (2, -1, 0), (2, 0, 0), (2, 1, 0), (2, 2, 0)])

## feature

In [32]:
from mlelec.utils.metatensor_utils import labels_where
from metatensor import Labels
from mlelec.features.acdc import twocenter_hermitian_features, single_center_features, pair_features, twocenter_hermitian_features_periodic
from mlelec.utils.twocenter_utils import map_targetkeys_to_featkeys
from mlelec.features.acdc import twocenter_features_periodic_NH
from mlelec.features.acdc_utils import _pca 



In [33]:
hypers = {'cutoff': 4,
          'max_radial':8, 
          'max_angular':4,
          'atomic_gaussian_width':0.3,
          'center_atom_weight':1,
          "radial_basis": {"Gto": {}},
          "cutoff_function": {"ShiftedCosine": {"width": 0.1}},
}

In [34]:
import metatensor

In [35]:
# two center features used for all translations except the 0,0,0 translation
rhoij = pair_features(frames, hypers, order_nu=[1,1],  all_pairs=True, mic=False, lcut=2, max_shift=[2,3,1], both_centers=True,device=device)# device = "cpu")

/home/hanna/data/Hamiltonian/jigyasa/mlelec/fhiaims/mlelec/features/acdc.py:136: UserWarning: Using cutoff 7.0 for all pairs feature
  warnings.warn(


In [36]:
blocks = []
# normalizing features - SKIP if needed 
# rhoij_norm = {tuple(t):[] for t in desired_shifts}
# for i, (k,b) in enumerate(rhoij.items()):
#       for t in desired_shifts[:]:
#             tlab, tidx = labels_where(b.samples, selection=Labels(names=["cell_shift_a", "cell_shift_b", "cell_shift_c"], values=np.array([t]).reshape(1,3)), return_idx=True)
#             rhoij_norm[tuple(t)].append(torch.linalg.norm(b.values[tidx])**2)

# for t in desired_shifts:
#       rhoij_norm[tuple(t)] =torch.sum(torch.tensor(rhoij_norm[tuple(t)])) 

for i, (k,b) in enumerate(rhoij.items()):
      # tlab, tidx = labels_where(b.samples, selection=Labels(names=["cell_shift_a", "cell_shift_b", "cell_shift_c"], values=np.array([t]).reshape(1,3)), return_idx=True)

      # b.values[tidx] = b.values[tidx]/torch.sqrt(rhoij_norm[tuple(t)]+1e-6) # dividing by norm of features, SKIP if needed 

      slab, sidx = labels_where(b.samples, selection=Labels(names=["cell_shift_a", "cell_shift_b", "cell_shift_c"], values=np.array(desired_shifts[:]).reshape(-1,3)), return_idx=True) # only retain tranlsations that we want - DONT SKIP 


      blocks.append(TensorBlock(values = b.values[sidx],
                              components = b.components,
                        samples = Labels(names = b.samples.names, values=np.asarray(b.samples.values[sidx])),
                              properties = b.properties)
                        )
rhoij = TensorMap(keys=rhoij.keys, blocks=blocks)
        
# One can PCA the features if desired 
# rhoij_pca = _pca(rhoij)

In [37]:
hfeat = twocenter_features_periodic_NH(single_center=None, pair= rhoij) # take the pair feature and create features corresponding to different blocks of the hamiltonian 

In [38]:
#hfeat[0]#.values

In [39]:
rhonui = single_center_features(frames, hypers, order_nu=3, lcut=2, max_shift=[2,3,1], feature_names = hfeat.property_names,device=device)
rhoij_zero_shift = pair_features(frames, hypers, order_nu=[1,1],  both_centers=True, all_pairs=True, lcut=2, max_shift=[1,1,1], mic=False, feature_names = rhonui.property_names, device=device)
# pair = rhoij
blocks = []
for i, (key, block) in enumerate(rhoij_zero_shift.items()):
    samples_lab, idx = labels_where(block.samples, selection=Labels(["cell_shift_a", "cell_shift_b", "cell_shift_c"], values=np.asarray([[0,0,0]]).reshape(-1, 3)), return_idx=True)
    samples_lab =  Labels(samples_lab.names[:3], values=samples_lab.values[:,:3])
    tb = TensorBlock(
        samples=samples_lab,
        values=block.values[idx],
        properties=block.properties,
        components = block.components
    )
    blocks.append(tb)
rhoij_zero_shift = TensorMap(keys = rhoij_zero_shift.keys, blocks=blocks)
hfeat0 = twocenter_features_periodic_NH(single_center= rhonui, pair = rhoij_zero_shift, shift=[0,0,0]) # take the pair feature and create features corresponding to different blocks of the hamiltonian
hfeat0 = hfeat0.keys_to_samples('cell_shift_a')
hfeat0 = hfeat0.keys_to_samples('cell_shift_b')
hfeat0 = hfeat0.keys_to_samples('cell_shift_c')
#hfeat0 = hfeat0.remove_samples('cell_shift_a_MIC')
#hfeat0 = hfeat0.keys_to_samples('cell_shift_b_MIC')
#hfeat0 = hfeat0.keys_to_samples('cell_shift_c_MIC')

/home/hanna/data/Hamiltonian/jigyasa/mlelec/fhiaims/mlelec/features/acdc.py:136: UserWarning: Using cutoff 2.0 for all pairs feature
  warnings.warn(


In [40]:
#hfeat0.keys.names[:-3]
rhoij_zero_shift
hfeat0[0].samples

Labels(
    structure  center  neighbor  cell_shift_a  cell_shift_b  cell_shift_c
        0        0        0           0             0             0
        0        1        1           0             0             0
        1        0        0           0             0             0
        1        1        1           0             0             0
        2        0        0           0             0             0
        2        1        1           0             0             0
)

In [41]:

    # sorted_blocks=[]
    # for k,b in hfeat0.items():
    #     #w=metatensor.sort_block(b, axes=[axis])
    #     #print(w.samples.values)
    #     sorted_blocks.append(b.copy())
        
    # hfeat0_2=TensorMap(Labels(hfeat0.keys.names[:-3],hfeat0.keys.values[:,:-3]), sorted_blocks)

In [42]:
hfeat0.keys

Labels(
    order_nu  inversion_sigma  spherical_harmonics_l  species_center  species_neighbor  block_type
       3             1                   0                  6                6              0
       3             1                   1                  6                6              0
       3             1                   2                  6                6              0
       3            -1                   1                  6                6              0
       3            -1                   2                  6                6              0
       3            -1                   0                  6                6              0
       3             1                   0                  6                6              1
       3             1                   0                  6                6              -1
       3             1                   1                  6                6              1
       3             1                   1    

In [43]:
hfeat0.property_names == hfeat.property_names

True

In [44]:
from metatensor.operations import join 
hfeat = join([hfeat, hfeat0],axis="properties")

In [45]:
print(target[0])
print(hfeat[0])
print(hfeat0[0])

TensorBlock
    samples (72): ['structure', 'center', 'neighbor', 'cell_shift_a', 'cell_shift_b', 'cell_shift_c']
    components (1): ['M']
    properties (1): ['value']
    gradients: None
TensorBlock
    samples (72): ['structure', 'center', 'neighbor', 'cell_shift_a', 'cell_shift_b', 'cell_shift_c']
    components (1): ['spherical_harmonics_m']
    properties (18944): ['tensor', 'species_neighbor_1_a', 'n_1_a', 'k_3', 'species_neighbor_1_a_b', 'n_1_a_b', 'k_2_b', 'species_neighbor_1_b_b', 'n_1_b_b', 'l_2_b', 'l_3']
    gradients: None
TensorBlock
    samples (6): ['structure', 'center', 'neighbor', 'cell_shift_a', 'cell_shift_b', 'cell_shift_c']
    components (1): ['spherical_harmonics_m']
    properties (11776): ['species_neighbor_1_a', 'n_1_a', 'k_3', 'species_neighbor_1_a_b', 'n_1_a_b', 'k_2_b', 'species_neighbor_1_b_b', 'n_1_b_b', 'l_2_b', 'l_3']
    gradients: None


## train

In [46]:
from mlelec.models.linear import LinearModelPeriodic
from mlelec.utils.plot_utils import plot_hamiltonian 

In [47]:
# target[0].samples.values.shape, hfeat[0].samples.values.shape
# target, hfeat

In [48]:
def sort_tmap(tmap, axis='samples'):
    sorted_blocks=[]
    for k,b in tmap.items():
        w=metatensor.sort_block(b, axes=[axis])
        #print(w.samples.values)
        sorted_blocks.append(w)
    return TensorMap(tmap.keys, sorted_blocks)


In [49]:
shfeat=sort_tmap(hfeat)
starget=sort_tmap(target)

In [50]:
#shfeat[0].samples.values

In [51]:
model = LinearModelPeriodic(shfeat, starget, frames, orbs,
                            cell_shifts= list(filtered_matrices.keys()), 
                            nhidden=16, nlayers=2, device=device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
# optimizer= torch.optim.SGD(model.parameters(), lr=1e-3)
#optimizer = torch.optim.LBFGS(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=100, verbose=True)


def train(model, target_blocks, nepochs=10, optimizer=None, scheduler=None, log_interval=10):
    losses = []
    for i in range(nepochs):
        # def closure():
        #     optimizer.zero_grad()
        #     rmat = model.forward()
        #     loss = 0
        #     for s in rmat:            
        #         loss += torch.linalg.norm(rmat[s]-matrices[tuple(s)])**2
        #     # loss = torch.linalg.norm(rmat - matrices)**2
        #     # loss.backward()
        #     return loss

        # optimizer.step(closure)
        # loss = closure()

    #---ADAM---- 
        optimizer.zero_grad()
        rmat = model.forward()
        loss = 0
        for s in rmat:      
            for (key, block) in rmat[s].items():
                loss += torch.linalg.norm(block.values-target_blocks[tuple(s)].block(key).values)**2      
            # loss += torch.sum((rmat[s]-matrices[tuple(s)])**2)


        loss.backward()
        optimizer.step()
        # scheduler.step(loss)
        losses.append(loss.item())
        if i%log_interval ==0:
            print(i, loss.item())
    return losses, model, optimizer


[(0, 0, 0), (0, 1, 0), (0, 2, 0), (1, -2, 0), (1, -1, 0), (1, 0, 0), (1, 1, 0), (1, 2, 0), (2, -1, 0), (2, 0, 0), (2, 1, 0), (2, 2, 0)] 12


/home/hanna/Software/venv/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


In [52]:
#target_coupled_blocks

In [ ]:
losses, model, optimizer = train(model, target_coupled_blocks, nepochs=6000, optimizer=optimizer)

0 696.0807495117188
10 661.3245849609375
20 609.5765991210938
30 541.9962158203125
40 452.3999938964844
50 333.7804260253906
60 201.03555297851562
70 86.84956359863281
80 29.21963882446289
90 23.165884017944336
100 22.502246856689453
110 17.614295959472656


In [ ]:
model

In [ ]:
plt.loglog(losses)

In [ ]:
mblocks[(0,0,0)][0].values

In [ ]:
mblocks = model.forward()
rmat = {}
for s in model.cell_shifts:
    rmat[tuple(s)] = _to_matrix(_to_uncoupled_basis(mblocks[tuple(s)], device=device), frames = frames, orbitals=orbs,  NH=True,device=device)  # DONT FORGET NH=True      


In [ ]:
mdict = {"mdict": model.state_dict(), "optimizer": optimizer.state_dict(), "epoch":4000}

In [ ]:
torch.save(mdict, "rotated_model.pt")
#model.load_state_dict(torch.load("rotated_model.pt"))
#model.eval()

In [ ]:
# error on individual translations
for translation in model.cell_shifts: 
    print(translation, torch.linalg.norm(rmat[tuple(translation)]-matrices[tuple(translation)]).detach().cpu().numpy())

In [ ]:
fig, ax, mappable = plot_hamiltonian((matrices[(0,-2,0)]).detach().cpu()[0], plot_abs = True)
fig.colorbar(mappable, ax=ax)
fig.tight_layout()
# fig.savefig('/home/nigam/act010.png', dpi = 400, bbox_inches = 'tight')

## TODO: try diff loss functions 

## band structure

In [ ]:
translated_ovls[(0,0,0)].shape

In [ ]:
translated_ovls=dict([(tuple(all_shifts[0][i]),ovls[:,i]) for i in range(len(all_shifts[0])) if tuple(all_shifts[0][i]) in target_coupled_blocks.keys() ])

In [ ]:
ovls = np.load('dataset/periodic/rotation/realspaceOvl_nonsym.npz'.format(filename), allow_pickle=True)
ovls=[ovls['arr_{}'.format(i)] for i in range(len(ovls))]
print(ovls[0].shape)

In [ ]:
cell_shifts.shape,ovls[0].shape

In [ ]:
translated_ovls=dict([(tuple(all_shifts[0][i]),np.array(ovls)[:,i]) for i in range(len(all_shifts[0])) if tuple(all_shifts[0][i]) in target_coupled_blocks.keys() ])

In [ ]:
len(translated_ovls.keys())

In [ ]:
rhams=np.zeros((len(rmat.keys())*2-1,*rmat[(0,0,0)].shape))
#pyscffock=np.zeros((len(rmat.keys())*2-1,*rmat[(0,0,0)].shape))
cell_shifts=np.zeros((len(rmat.keys())*2-1,3))
ovls=np.zeros((len(rmat.keys())*2-1,*rmat[(0,0,0)].shape))
for n,key in enumerate(rmat.keys()):
    cell_shifts[n]=key
    rhams[n]=rmat[key].detach().numpy()
    ovls[n]=translated_ovls[key]
    #pyscffock[n]=rotated_dataset.matrices_translation[key]
    #print(rotated_dataset.matrices_translation[key].shape)
    
    cell_shifts[-n]=tuple([-key[0],-key[1],-key[2]])
    rhams[-n]=np.transpose(rmat[key].detach().numpy(), axes=(0,2,1))
    ovls[-n]=np.transpose(translated_ovls[key], axes=(0,2,1))
    #pyscffock[-n]=np.transpose(rotated_dataset.matrices_translation[key], axes=(0,2,1))

print(rhams.shape,cell_shifts.shape)

In [ ]:
rhams=np.transpose(rhams, axes=(1,0,2,3))
ovls=np.transpose(ovls, axes=(1,0,2,3))
#pyscffock=np.transpose(pyscffock, axes=(1,0,2,3))

print(rhams.shape)#,ovls.shape,pyscffock.shape)

In [ ]:
from mlelec.utils.plot_bands_from_realspace import plot_bandstructure,plot_multiple_bandstructures

In [ ]:
frames[struc],rhams[struc].shape,ovls[struc].shape, cell_shifts.shape

In [ ]:
struc=0
npoints=50
symmpoints=[[0.0,0.0,0.0], [0.5,0.0,0.0],[0.33333,0.33333,0.0],[0,0,0]]
symmpoint_names=['Gamma','M','K','Gamma']
plot_bandstructure(frames[struc],rhams[struc],ovls[struc], cell_shifts,symmpoints, symmpoint_names, npoints)
plt.savefig('fhiaims.png', transparent=True)

In [ ]:
struc=0
npoints=50
symmpoints=[[0.0,0.0,0.0], [0.5,0.0,0.0],[0.33333,0.33333,0.0],[0,0,0]]
symmpoint_names=['Gamma','M','K','Gamma']
plot_bandstructure(frames[struc],focks[struc],ovls[struc], cell_shifts,symmpoints, symmpoint_names, npoints)
plt.savefig('learned.png', transparent=True)